In [9]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import pandas as pd
import time

In [2]:
#splinter dependencies
!pip install splinter
!pip install webdriver_manager

from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager



In [3]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

# URL of page to be scraped
url1 = 'https://mars.nasa.gov/news/'
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
baseimageurl2='https://www.jpl.nasa.gov'
url3='https://space-facts.com/mars/'
url4='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(url1)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\17347\.wdm\drivers\chromedriver\win32\87.0.4280.88]


In [10]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)



In [11]:
# Part 1:  collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later

# Retrieve page with the requests module (maybe dont need since using browser)
#response1 = requests.get(url1)

# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(browser.html, 'lxml')

In [12]:
#collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

title=soup.select_one('li.slide').find('div', class_="image_and_description_container").find('div', class_='content_title').get_text()
print(title)

NASA Extends Exploration for Two Planetary Science Missions


In [13]:
paragraph=soup.select_one('li.slide').find('div', class_="image_and_description_container").find('div', class_='article_teaser_body').get_text()
print(paragraph)

The missions – Juno and InSight – have each increased our understanding of our solar system, as well as spurred new sets of diverse questions.


In [14]:
# Part 2:  using splinter to to navigate the site and find the image url for the current Featured Mars Image 
#and assign the url string to a variable called `featured_image_url`.

#count to one second and then go to the next line (this is good practice in cases where there are multiple webpages within the same notebook)
time.sleep(1)

In [15]:
#need to go to the next URL for this part
browser.visit(url2)
soup = BeautifulSoup(browser.html, 'lxml')

In [16]:

featured_image_url=soup.find('div', class_="carousel_items").find('article', class_='carousel_item').get('style')
print(featured_image_url[23:-3])

/spaceimages/images/wallpaper/PIA20063-1920x1200.jpg


In [17]:
FINAL_image_url= (baseimageurl2+(featured_image_url[23:-3]))
FINAL_image_url



'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA20063-1920x1200.jpg'

In [ ]:
### another WAY to do this!!!

#images= soup.find('article', class_='carousel_item')['style']
#featured=images.split("'")
#featured_image ="https://www.jpl.nasa.gov/"+featured[1]
#print(featured_image)

In [18]:
#need to go to the next URL for this part
browser.visit(url3)
soup = BeautifulSoup(browser.html, 'lxml')

In [19]:
# this gives ALL items displayed as tables on the webpage (as a list)
Marsfacts = pd.read_html(url3)
Marsfacts

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [20]:
# in order to get just the first item in the list (ie the Mars facts table ONLY)
MarsFactsMain=Marsfacts[0]
MarsFactsMain

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [21]:
# this is still only a list-- need to Make a proper table, then convert back to a HTML table string (for Step 2 of homework)
MarsFactsMain=MarsFactsMain.rename(columns={0: "Description", 1: "Mars"})
MarsFactsMain


,Description,Mars
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [22]:
# Convert back to HTML string (this variable can just be inserted into html file from flask)
Factstablehtml=MarsFactsMain.to_html()
Factstablehtml


'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Mars</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium B

In [ ]:
## these steps ensure that if the original table/images (on original website) are updated, then they update on 'our' website also!! 
## if, only copied over the from the website, then updates not reflected.  

In [23]:
#need to go to the next URL for this part
browser.visit(url4)
soup = BeautifulSoup(browser.html, 'lxml')

In [26]:
# make a list of all the class="item" where the four links are located
hemispherehtml=soup.find_all('div', class_="item")
print(len(hemispherehtml))

4


In [35]:
##  this block is just to see how this data can be extracted for the first one, before trying the loop
#link=hemispherehtml[0]
#link

#imagelink=link.find("a")['href']
#imagetitle=link.find("h3").get_text()
#print(imagelink, imagetitle)

/search/map/Mars/Viking/cerberus_enhanced Cerberus Hemisphere Enhanced


In [38]:
# first, create a list where the dictionaries of each component can be saved

hemispheresList=[]

# create a loop so that it goes through and make a dictionary for each
for link in hemispherehtml:
    hemispheredict={}
    
    
    # define the stuff that needs to go in each dictionary
    imagelink=link.find("a")['href']
    imagetitle=link.find("h3").get_text()
    
    hemispheredict["ImageTitle"]=imagetitle
    hemispheredict["ImageLink"]=imagelink
    
    ## now, put all disctionaries above in the List
    
    hemispheresList.append(hemispheredict)    

print(hemispheresList)

[{'ImageTitle': 'Cerberus Hemisphere Enhanced', 'ImageLink': '/search/map/Mars/Viking/cerberus_enhanced'}, {'ImageTitle': 'Schiaparelli Hemisphere Enhanced', 'ImageLink': '/search/map/Mars/Viking/schiaparelli_enhanced'}, {'ImageTitle': 'Syrtis Major Hemisphere Enhanced', 'ImageLink': '/search/map/Mars/Viking/syrtis_major_enhanced'}, {'ImageTitle': 'Valles Marineris Hemisphere Enhanced', 'ImageLink': '/search/map/Mars/Viking/valles_marineris_enhanced'}]
